In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error, explained_variance_score, max_error, d2_absolute_error_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from surprise import SVD, Reader, Dataset
from collections import defaultdict
import gzip
from surprise import SVDpp
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import GridSearchCV

### DATASET

In [94]:
recipe = pd.read_csv("PP_recipes.csv")
recipe = recipe[["id", "name_tokens", "ingredient_tokens", "steps_tokens"]]
recipe.head()

,id,name_tokens,ingredient_tokens,steps_tokens
0,424415,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161..."
1,146223,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393..."
2,312329,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66..."
3,74301,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4..."
4,76272,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,..."


In [95]:
user = pd.read_csv("PP_users.csv")
user.head()

,u,techniques,items,n_items,ratings,n_ratings
0,0,"[8, 0, 0, 5, 6, 0, 0, 1, 0, 9, 1, 0, 0, 0, 1, ...","[1118, 27680, 32541, 137353, 16428, 28815, 658...",31,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, ...",31
1,1,"[11, 0, 0, 2, 12, 0, 0, 0, 0, 14, 5, 0, 0, 0, ...","[122140, 77036, 156817, 76957, 68818, 155600, ...",39,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...",39
2,2,"[13, 0, 0, 7, 5, 0, 1, 2, 1, 11, 0, 1, 0, 0, 1...","[168054, 87218, 35731, 1, 20475, 9039, 124834,...",27,"[3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, ...",27
3,3,"[498, 13, 4, 218, 376, 3, 2, 33, 16, 591, 10, ...","[163193, 156352, 102888, 19914, 169438, 55772,...",1513,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, ...",1513
4,4,"[161, 1, 1, 86, 93, 0, 0, 11, 2, 141, 0, 16, 0...","[72857, 38652, 160427, 55772, 119999, 141777, ...",376,"[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, 5.0, ...",376


In [96]:
reviews = pd.read_csv("RAW_interactions.csv")
reviews = reviews.drop("date", axis=1)
reviews.head()

,user_id,recipe_id,rating,review
0,38094,40893,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,4,This worked very well and is EASY. I used not...
3,126440,85009,5,I made the Mexican topping and took it to bunk...
4,57222,85009,5,"Made the cheddar bacon topping, adding a sprin..."


In [113]:
reviews.shape

(1132367, 4)

In [97]:
reviews_small = reviews.iloc[:100000]


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Map user_id and recipe_id to unique integers
user_encoder = LabelEncoder()
recipe_encoder = LabelEncoder()

reviews_small['user_idx'] = user_encoder.fit_transform(reviews_small['user_id'])
reviews_small['recipe_idx'] = recipe_encoder.fit_transform(reviews_small['recipe_id'])

print(reviews_small[['user_id', 'user_idx', 'recipe_id', 'recipe_idx']].head())


   user_id  user_idx  recipe_id  recipe_idx
0    38094      1204      40893        2187
1  1293707     24101      40893        2187
2     8937       150      44394        2362
3   126440      4629      85009        4609
4    57222      2122      85009        4609


C:\Users\Owner\AppData\Local\Temp\ipykernel_12836\2548817215.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_small['user_idx'] = user_encoder.fit_transform(reviews_small['user_id'])
C:\Users\Owner\AppData\Local\Temp\ipykernel_12836\2548817215.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_small['recipe_idx'] = recipe_encoder.fit_transform(reviews_small['recipe_id'])


### TEXT EMBEDDING

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np


reviews_only = reviews_small["review"].fillna("").str.lower().str.replace(r"[^\w\s]", "", regex=True)

sentences = reviews_only.apply(lambda x: str(x).split())

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_embeddings = {word: word2vec_model.wv[word] for word in word2vec_model.wv.index_to_key}

def get_sentence_embedding(sentence):
    words = sentence.split()
    embeddings = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

sentence_embeddings = reviews_only.apply(get_sentence_embedding)


### PREPARING TRAIN, TEST SET

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

data_df = pd.DataFrame({
    'user_id': reviews_small['user_id'],
    'recipe_id': reviews_small['recipe_id'],
    'review_embedding': list(sentence_embeddings),
    'rating': reviews_small['rating']
})

train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)

# Use Categorical to encode users and recipes, ensuring consistent mapping
train_df['user_idx'] = pd.Categorical(train_df['user_id']).codes
train_df['recipe_idx'] = pd.Categorical(train_df['recipe_id']).codes

test_df['user_idx'] = pd.Categorical(
    test_df['user_id'], categories=train_df['user_id'].unique()
).codes

test_df['recipe_idx'] = pd.Categorical(
    test_df['recipe_id'], categories=train_df['recipe_id'].unique()
).codes

test_df['user_idx'] = test_df['user_idx'].replace(-1, len(train_df['user_id'].unique()))
test_df['recipe_idx'] = test_df['recipe_idx'].replace(-1, len(train_df['recipe_id'].unique()))

X_train_user_idx = train_df['user_idx'].values
X_train_recipe_idx = train_df['recipe_idx'].values
X_train_review_embedding = np.array(train_df['review_embedding'].tolist())
y_train = train_df['rating'].values

X_test_user_idx = test_df['user_idx'].values
X_test_recipe_idx = test_df['recipe_idx'].values
X_test_review_embedding = np.array(test_df['review_embedding'].tolist())
y_test = test_df['rating'].values

print(f"Max user index in training: {max(X_train_user_idx)}, Unique users in training: {len(np.unique(X_train_user_idx))}")
print(f"Max recipe index in training: {max(X_train_recipe_idx)}, Unique recipes in training: {len(np.unique(X_train_recipe_idx))}")


Max user index in training: 32373, Unique users in training: 32374
Max recipe index in training: 17861, Unique recipes in training: 17862


In [ ]:
import torch
import numpy as np

user_idx_train_tensor = torch.tensor(X_train_user_idx, dtype=torch.long)
recipe_idx_train_tensor = torch.tensor(X_train_recipe_idx, dtype=torch.long)
review_embedding_train_tensor = torch.tensor(np.array(X_train_review_embedding.tolist()), dtype=torch.float32)
rating_train_tensor = torch.tensor(y_train, dtype=torch.float32)

user_idx_test_tensor = torch.tensor(X_test_user_idx, dtype=torch.long)
recipe_idx_test_tensor = torch.tensor(X_test_recipe_idx, dtype=torch.long)
review_embedding_test_tensor = torch.tensor(np.array(X_test_review_embedding.tolist()), dtype=torch.float32)
rating_test_tensor = torch.tensor(y_test, dtype=torch.float32)

### MODEL

In [ ]:
import torch
import torch.nn as nn

class RatingPredictionModel(nn.Module):
    def __init__(self, num_users, num_recipes, review_embedding_dim, embedding_dim, dropout_rate=0.1):
        super(RatingPredictionModel, self).__init__()
        
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.recipe_embedding = nn.Embedding(num_recipes, embedding_dim)
        
        self.fc1 = nn.Linear(embedding_dim * 2 + review_embedding_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        
        self.dropout = nn.Dropout(dropout_rate)

        self.relu = nn.ReLU()

    def forward(self, user_idx, recipe_idx, review_embedding):
        user_vec = self.user_embedding(user_idx)
        recipe_vec = self.recipe_embedding(recipe_idx)
        
        x = torch.cat([user_vec, recipe_vec, review_embedding], dim=1)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x


In [ ]:
num_users = len(user_encoder.classes_) + 1
num_recipes = len(user_encoder.classes_) + 1
review_embedding_dim = review_embedding_train_tensor.shape[1]
embedding_dim = 30

model = RatingPredictionModel(num_users, num_recipes, review_embedding_dim, embedding_dim)


### TRAINING

In [ ]:
epochs = 170
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
for epoch in range(epochs):
    model.train()  # Ensure the model is in training mode
    optimizer.zero_grad()  # Zero gradients before the backward pass
    
    # Forward pass: Get predictions using the training set
    predictions_train = model(user_idx_train_tensor, recipe_idx_train_tensor, review_embedding_train_tensor)
    
    # Compute loss for the training set
    loss = criterion(predictions_train.squeeze(), rating_train_tensor)
    
    # Backward pass: Compute gradients
    loss.backward()
    
    # Update model weights
    optimizer.step()

    # Print MSE after each epoch
    mse = loss.item()  # MSE is the loss value (MSELoss returns a scalar)
    print(f"Epoch {epoch+1}/{epochs}, Training MSE: {mse}")

Epoch 1/170, Training MSE: 20.171756744384766
Epoch 2/170, Training MSE: 19.44502830505371
Epoch 3/170, Training MSE: 18.734329223632812
Epoch 4/170, Training MSE: 18.02838706970215
Epoch 5/170, Training MSE: 17.30881690979004
Epoch 6/170, Training MSE: 16.5572566986084
Epoch 7/170, Training MSE: 15.758737564086914
Epoch 8/170, Training MSE: 14.902666091918945
Epoch 9/170, Training MSE: 13.982306480407715
Epoch 10/170, Training MSE: 12.995048522949219
Epoch 11/170, Training MSE: 11.942282676696777
Epoch 12/170, Training MSE: 10.829741477966309
Epoch 13/170, Training MSE: 9.668360710144043
Epoch 14/170, Training MSE: 8.474952697753906
Epoch 15/170, Training MSE: 7.2736592292785645
Epoch 16/170, Training MSE: 6.096723556518555
Epoch 17/170, Training MSE: 4.985211372375488
Epoch 18/170, Training MSE: 3.98888897895813
Epoch 19/170, Training MSE: 3.163602352142334
Epoch 20/170, Training MSE: 2.5606741905212402
Epoch 21/170, Training MSE: 2.1968138217926025
Epoch 22/170, Training MSE: 2.0207

In [185]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Disable gradient computation
    # Convert test inputs to tensors
    user_idx_tensor = torch.tensor(X_test_user_idx, dtype=torch.long)
    recipe_idx_tensor = torch.tensor(X_test_recipe_idx, dtype=torch.long)
    review_embedding_tensor = torch.tensor(X_test_review_embedding, dtype=torch.float)

    # Get predictions
    y_pred = model(user_idx_tensor, recipe_idx_tensor, review_embedding_tensor)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE: {rmse}")


RMSE: 1.364762876585554


In [201]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Function to train the model
def train_model(weight_decay, train_loader, val_loader, num_epochs=10):
    # Initialize model
    model = RatingPredictionModel(num_users, num_recipes, review_embedding_dim, embedding_dim)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        
        for user_idx, recipe_idx, review_embedding, ratings in train_loader:            
            optimizer.zero_grad()
            predictions = model(user_idx, recipe_idx, review_embedding)
            loss = criterion(predictions.squeeze(), ratings)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        # Validation step
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for user_idx, recipe_idx, review_embedding, ratings in val_loader:
                predictions = model(user_idx, recipe_idx, review_embedding)
                val_loss += criterion(predictions.squeeze(), ratings).item()
        
        print(f"Weight Decay: {weight_decay}, Epoch: {epoch+1}, "
              f"Train Loss: {epoch_loss/len(train_loader)}, "
              f"Val Loss: {val_loss/len(val_loader)}")
    
    return model

# Define weight decay values to test
weight_decay_values = [0.0, 0.1, 0.01, 0.001, 0.0001]

# Prepare DataLoaders
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(user_idx_train_tensor, recipe_idx_train_tensor, review_embedding_train_tensor, y_train_tensor)
val_dataset = TensorDataset(user_idx_test_tensor, recipe_idx_test_tensor, review_embedding_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Train models with different weight decay values
trained_models = {}
for wd in weight_decay_values:
    print(f"\nTraining with weight_decay={wd}")
    trained_models[wd] = train_model(wd, train_loader, val_loader)



Training with weight_decay=0.0
Weight Decay: 0.0, Epoch: 1, Train Loss: 1.7441638114929199, Val Loss: 1.4882883456187508
Weight Decay: 0.0, Epoch: 2, Train Loss: 1.4120799545884133, Val Loss: 1.4163859379938997
Weight Decay: 0.0, Epoch: 3, Train Loss: 1.2776583253622056, Val Loss: 1.3950434180494315
Weight Decay: 0.0, Epoch: 4, Train Loss: 1.163794349384308, Val Loss: 1.3873980592805357
Weight Decay: 0.0, Epoch: 5, Train Loss: 1.0576284823894502, Val Loss: 1.4628228221457606
Weight Decay: 0.0, Epoch: 6, Train Loss: 0.9301262266993523, Val Loss: 1.6144630573808956
Weight Decay: 0.0, Epoch: 7, Train Loss: 0.7884988143324853, Val Loss: 1.6357947663186838
Weight Decay: 0.0, Epoch: 8, Train Loss: 0.6510674505829811, Val Loss: 1.8393964933130307
Weight Decay: 0.0, Epoch: 9, Train Loss: 0.5212625410020352, Val Loss: 1.8063482805943718
Weight Decay: 0.0, Epoch: 10, Train Loss: 0.4145885623216629, Val Loss: 2.0397689938545227

Training with weight_decay=0.1
Weight Decay: 0.1, Epoch: 1, Train L